# Feature Engineering

## 1. Setup

In [1]:
import pandas as pd
import numpy as np
from summarytools import dfSummary

from env_setup import *
from functions.t_test import *

pd.set_option('display.max_columns', None)

In [2]:
# Load data
df_raw = pd.read_csv(fr"{dataout}//{dataset}_EDA.csv")
df_raw.head()
df_ori = df_raw.copy()

## 2. Feature Engineering

In [3]:
# Split categorical and numeric
l_cols_cat = [i for i in df_ori if df_ori[i].dtype==object]
l_cols_num = [i for i in df_ori if df_ori[i].dtype!=object and i != 'Churned']

print(f"Categorical columns: {', '.join(l_cols_cat)}")
print(f"Numeric columns: {', '.join(l_cols_num)}")

Categorical columns: Gender, Location, TransactionType
Numeric columns: Age, Income, Tenure, TransactionDate, TransactionAmount, NumSupportCalls, NumComplaints


### 2.1. Categorical One Hot Encoding
Apply one-hot encoding to categorical variables for modelling

In [4]:
# Summary Statistics for Categorical
## Encode categorical
df_ori_encoded = pd.get_dummies(df_ori, columns=l_cols_cat)
l_cols_encoded = [i for i in df_ori_encoded.columns if any(x in i for x in l_cols_cat)]

for col in l_cols_encoded:
    df_ori_encoded[col] = np.where(df_ori_encoded[col]==True, 1, 0)

stats_cat = df_ori_encoded.groupby('Churned')[l_cols_encoded].sum().reset_index()

## Scale
stats_cat_scaled = stats_cat.copy()
for col in l_cols_encoded:
    stats_cat_scaled[col] = np.where(stats_cat_scaled['Churned']==0, 
                                    stats_cat_scaled[col] / float(df_ori_encoded[df_ori_encoded['Churned']==0]['Churned'].count()),
                                    stats_cat_scaled[col] / float(df_ori_encoded[df_ori_encoded['Churned']==1]['Churned'].count())
    )
stats_cat_scaled

,Churned,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban,TransactionType_Purchase,TransactionType_Refund,TransactionType_Subscription Renewal,TransactionType_Support Fee,TransactionType_Upgrade
0,0,0.503888,0.496112,0.199020,0.300483,0.500497,0.200235,0.199811,0.199318,0.200137,0.200498
1,1,0.498657,0.501343,0.195221,0.306133,0.498646,0.199729,0.198015,0.200229,0.201687,0.200340


### 2.2. Numerical Binning
Apply binning to numerical variables.

Mostly not necessary as bands are not distinct enough but just in case needed for modelling.

In [5]:
# Create Bins
df_ori_binned = df_ori.copy()

d_bins = {
    'Age': list(range(0, 80, 10)), 
    'Income': list(range(0, 200_000, 10_000)), 
    'Tenure': [0, 5], # Hypothesis test cut off = 5
    'TransactionDate': list(range(0, 390, 30)),
    'TransactionAmount': list(range(0, 600, 100)), 
    # 'NumSupportCalls': [], # Not required - too small and no major difference in hypothesis test
    # 'NumComplaints': [] # Not required - too small and no major difference in hypothesis test
}

for i,j in d_bins.items():
    df_ori_binned[f'{i}_bin'] = pd.cut(df_ori_binned[i], bins=j)

# Fix nans
df_ori_binned['Tenure_bin'] = df_ori_binned['Tenure_bin'].cat.add_categories('>5')
df_ori_binned.loc[df_ori_binned['Tenure_bin'].isna(), 'Tenure_bin'] = '>5'

In [6]:
# View Statistics
dfSummary(df_ori_binned[[f'{i}_bin' for i in d_bins.keys()]])

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Age_bin[category],"1. (30, 40]2. (40, 50]3. (50, 60]4. (20, 30]5. (60, 70]6. (10, 20]","195,706 (19.6%)192,682 (19.3%)190,159 (19.1%)189,473 (19.0%)172,166 (17.3%)57,445 (5.8%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAACKCAYAAAC96ziyAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACVklEQVR4nO3bQYoaQQCG0TLIkNkoIniYOUQOm0N4GxF7kRlmYxY5wAgjXwfqvQuUi4+Sbvrf3O/3AYUfa/8A5iE2MmIjIzYymzHG6xjjZYWzP+/3+/sK57KS7fF4/LXb7Q71wcuyXDebzW/BzWO72+0Ob29v7/v9/qM69Ha7/Tyfz4fL5fIyxhDbJLZjjLHf7z9Op9Of+OzX+DxW5gGBjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNzHaMf5/8lIfW5/F/2C7Lcj2fz4cRf/KzLMt1jPFZnsm6fBZOZmOkTMXNRsbghYzBCxmDFzJe6pIRGxmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxnrKjLWVWRsEMhYV5Fxs5GxriJjXUXGuoqMl7pkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEbGuoqMdRUZGwQy1lVk1rrZ3GoTWmVdZVk1p3xdZVk1r7XWVZZVE/JSl4zYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjUw+eDF2mdcqgxdjlzn5LJyMwQsZDwhk6r9Rf58TS9dVVlVzy9ZVVlXU6yqrqol5QCAjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiOTrausqkjXVVZVc/NZOBnrKjKP3mxuJL7tocGLoQrP8OXgxVCFZ3l08GKowrd5qUtGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGS+XFdZRfEsD62rrKJ4BhsEMtZVZDwgkBEbGbGR+QtvGEOmKMmnxgAAAABJRU5ErkJggg=="">",0(0.0%)
2,Income_bin[category],"1. (70000, 80000]2. (50000, 60000]3. (90000, 100000]4. (60000, 70000]5. (80000, 90000]6. (40000, 50000]7. (20000, 30000]8. (30000, 40000]9. (10000, 20000]","126,797 (12.7%)126,121 (12.6%)125,141 (12.5%)125,025 (12.5%)124,929 (12.5%)124,920 (12.5%)123,380 (12.4%)121,286 (12.2%)32 (0.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAADPCAYAAADifqNtAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADFUlEQVR4nO3cwU0UcRyG4d+QDeplN4SEErQHirBYi9gi7IEQ5iDIZT140kQ4OL4k6/MU8M8e3swwk/lYTqfTQOHirX8A/w+xkREbGbGRERsZsZERG5llZj7MzOWGZz6fTqfHDc/jTOyur68/7/f7q60OXNf1flmWL4Ljd7v9fn91e3v7eDgcnv72sIeHh/fH4/Hq7u7ucmbExi92MzOHw+Hp5ubm20ZnftjoHM6MBwQyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyu5mfnwZtcdhW53Ceduu63h+Px6vZ6NOgdV3vZ+Z5i7M4Lz4LJ7P4Xx9UXNnIGLyQMXghY/BCxktdMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MgYvZAxeyPgsnIzBCxkPCGS2vo26hfJHm66rLKt4yWbrKssqXrP1usqyij/ygEBGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZms3WVZRWv2XRdZVnFS3wWTsa6iowrGxmDFzIGL2QMXsh4qUtGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEbG4IWMwQsZn4WTMXgh48pGxuCFjMELGYMXMl7qkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGesqMtZVZGwQyFhXkXFlI2NdRca6iox1FRkvdcmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MgYvJAxeCHjs3AyBi9kXNnIGLyQMXghY/BCxktdMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjY11FxrqKjA0CGesqMq5sZKyryFhXkbGuIuOlLhmxkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmDFzIGL2R8Fk7G4IWMBwQyF8uyfFqWxR/3/HMXM/NxZt699Q/h/LmNkhEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERuZi5n5OjPf3/qHcP4MXsi4jZIRGxmxkREbGbGRERsZsZH5AVzXs+MjkvOuAAAAAElFTkSuQmCC"">",0(0.0%)
3,Tenure_bin[category],"1. (0, 5]2. >5","556,204 (55.8%)441,427 (44.2%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjkuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8hTgPZAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABOklEQVR4nO3bQWrDMBRF0a8iQjORMAYvxovoYrsI7cYYa9CETJRBCgWjYfRk3HtW8DEXQ4KfK6UYoODM7Gpml96HnNCjlHLrfcSR+HEcv0IIQ+9DzibnvDrnvgnujw8hDPM832KM997HnMW2bZ8ppWFZlouZEdsvb2YWY7xP0/TT+5iTufY+4Gg+eh+A/4PYIENskCE2yBAbZIgNMsQGGWKDDLFBhtggQ2yQITbIEBtkiA0yxAYZb/b62K/3IWfC86zzOec1pTQYH/u9Vc55NbNH7zuOhMFLOwxedhxTPqjwZqvjrdQAU74KZnhtMOXbYYbXDlO+On6ZN8CfupAhNsgQG2SIDTLEBhligwyxQYbYIENskCE2yBAbZIgNMsQGGWKDDLFBhinfDs+iHaZ8Fczw2mDwUsfgpYEnCltq37tbH3oAAAAASUVORK5CYII="">",0(0.0%)
4,TransactionDate_bin[category],"1. (0, 30]2. (30, 60]3. (60, 90]4. (90, 120]5. (120, 150]6. (150, 180]7. (180, 210]8. (210, 240]9. (240, 270]10. (270, 300]11. other","287,270 (28.8%)173,643 (17.4%)129,981 (13.0%)102,680 (10.3%)81,885 (8.2%)64,758 (6.5%)51,220 (5.1%)39,624 (4.0%)29,524 (3.0%)20,596 (2.1%)16,450 (1.6%)","<img src = ""data:image/png;base64, iVBOR

In [9]:
# One Hot Encoding
df_ori_encoded_num = pd.get_dummies(df_ori_binned[[f'{i}_bin' for i in d_bins.keys()]], columns=[f'{i}_bin' for i in d_bins.keys()])

for col in df_ori_encoded_num.columns:
    df_ori_encoded_num[col] = np.where(df_ori_encoded_num[col]==True, 1, 0)

df_ori_encoded_num.head()

,"Age_bin_(0, 10]","Age_bin_(10, 20]","Age_bin_(20, 30]","Age_bin_(30, 40]","Age_bin_(40, 50]","Age_bin_(50, 60]","Age_bin_(60, 70]","Income_bin_(0, 10000]","Income_bin_(10000, 20000]","Income_bin_(20000, 30000]","Income_bin_(30000, 40000]","Income_bin_(40000, 50000]","Income_bin_(50000, 60000]","Income_bin_(60000, 70000]","Income_bin_(70000, 80000]","Income_bin_(80000, 90000]","Income_bin_(90000, 100000]","Income_bin_(100000, 110000]","Income_bin_(110000, 120000]","Income_bin_(120000, 130000]","Income_bin_(130000, 140000]","Income_bin_(140000, 150000]","Income_bin_(150000, 160000]","Income_bin_(160000, 170000]","Income_bin_(170000, 180000]","Income_bin_(180000, 190000]","Tenure_bin_(0, 5]",Tenure_bin_>5,"TransactionDate_bin_(0, 30]","TransactionDate_bin_(30, 60]","TransactionDate_bin_(60, 90]","TransactionDate_bin_(90, 120]","TransactionDate_bin_(120, 150]","TransactionDate_bin_(150, 180]","TransactionDate_bin_(180, 210]","TransactionDate_bin_(210, 240]","TransactionDate_bin_(240, 270]","TransactionDate_bin_(270, 300]","TransactionDate_bin_(300, 330]","TransactionDate_bin_(330, 360]","TransactionAmount_bin_(0, 100]","TransactionAmount_bin_(100, 200]","TransactionAmount_bin_(200, 300]","TransactionAmount_bin_(300, 400]","TransactionAmount_bin_(400, 500]"
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


## 3. Recombine

In [21]:
df_fe = pd.concat(
    [
        df_raw, # Original
        df_ori_encoded.drop(columns=['Age', 'Income', 'Tenure', 'TransactionDate', 'TransactionAmount', 'NumSupportCalls', 'NumComplaints', 'Churned']), # Categorical dummy
        df_ori_binned[[f'{i}_bin' for i in d_bins.keys()]], # Numeric binned
        df_ori_encoded_num, # Numeric binned dummy
    ],
    axis = 1)
df_fe.head()

,Age,Gender,Income,Tenure,Location,TransactionDate,TransactionType,TransactionAmount,NumSupportCalls,NumComplaints,Churned,Gender_Female,Gender_Male,Location_Rural,Location_Suburban,Location_Urban,TransactionType_Purchase,TransactionType_Refund,TransactionType_Subscription Renewal,TransactionType_Support Fee,TransactionType_Upgrade,Age_bin,Income_bin,Tenure_bin,TransactionDate_bin,TransactionAmount_bin,"Age_bin_(0, 10]","Age_bin_(10, 20]","Age_bin_(20, 30]","Age_bin_(30, 40]","Age_bin_(40, 50]","Age_bin_(50, 60]","Age_bin_(60, 70]","Income_bin_(0, 10000]","Income_bin_(10000, 20000]","Income_bin_(20000, 30000]","Income_bin_(30000, 40000]","Income_bin_(40000, 50000]","Income_bin_(50000, 60000]","Income_bin_(60000, 70000]","Income_bin_(70000, 80000]","Income_bin_(80000, 90000]","Income_bin_(90000, 100000]","Income_bin_(100000, 110000]","Income_bin_(110000, 120000]","Income_bin_(120000, 130000]","Income_bin_(130000, 140000]","Income_bin_(140000, 150000]","Income_bin_(150000, 160000]","Income_bin_(160000, 170000]","Income_bin_(170000, 180000]","Income_bin_(180000, 190000]","Tenure_bin_(0, 5]",Tenure_bin_>5,"TransactionDate_bin_(0, 30]","TransactionDate_bin_(30, 60]","TransactionDate_bin_(60, 90]","TransactionDate_bin_(90, 120]","TransactionDate_bin_(120, 150]","TransactionDate_bin_(150, 180]","TransactionDate_bin_(180, 210]","TransactionDate_bin_(210, 240]","TransactionDate_bin_(240, 270]","TransactionDate_bin_(270, 300]","TransactionDate_bin_(300, 330]","TransactionDate_bin_(330, 360]","TransactionAmount_bin_(0, 100]","TransactionAmount_bin_(100, 200]","TransactionAmount_bin_(200, 300]","TransactionAmount_bin_(300, 400]","TransactionAmount_bin_(400, 500]"
0,56,Female,91512,4,Urban,17,Support Fee,147.70,2,3,0,1,0,0,0,1,0,0,0,1,0,"(50, 60]","(90000, 100000]","(0, 5]","(0, 30]","(100, 200]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,56,Female,91512,4,Urban,19,Support Fee,90.61,2,3,0,1,0,0,0,1,0,0,0,1,0,"(50, 60]","(90000, 100000]","(0, 5]","(0, 30]","(0, 100]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,56,Female,91512,4,Urban,11,Purchase,180.82,2,3,0,1,0,0,0,1,1,0,0,0,0,"(50, 60]","(90000, 100000]","(0, 5]","(0, 30]","(100, 200]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,56,Female,91512,4,Urban,4,Refund,115.54,2,3,0,1,0,0,0,1,0,1,0,0,0,"(50, 60]","(90000, 100000]","(0, 5]","(0, 30]","(100, 200]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,56,Female,91512,4,Urban,15,Subscription Renewal,36.45,2,3,0,1,0,0,0,1,0,0,1,0,0,"(50, 60]","(90000, 100000]","(0, 5]","(0, 30]","(0, 100]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


## 4. Export

In [23]:
# Export datasets
df_fe.to_csv(fr"{dataout}//{dataset}_FE.csv", index=False)